<a href="https://colab.research.google.com/github/ChirathD/Notebooks/blob/master/siamese_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import shutil
from glob import glob
import os
from tqdm import tqdm

In [ ]:
from os import walk
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow 
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
import random

random.seed(40)

In [ ]:
!gdown --id 0B8-rUzbwVRk0c054eEozWG9COHM

In [ ]:
!unzip /content/Market-1501-v15.09.15.zip

In [ ]:
!rm /content/Market-1501-v15.09.15/bounding_box_train/Thumbs.db

In [ ]:
persons = os.listdir('/content/Market-1501-v15.09.15/bounding_box_train')

In [ ]:
pattern=re.compile(r'([-\d]+)_c(\d)')

In [ ]:
count = np.ones(len(persons))
for img_name in persons:
  id,cam = pattern.search(img_name).groups()
  
  count[int(id)] += 1  

k=0
for j in count:
  if(j<18 and j>15):
    k +=1

print(k)

In [ ]:
current_path = '/content/Market-1501-v15.09.15/bounding_box_train'
for person in tqdm(persons):
  if os.path.splitext(person)[1]== '.jpg':
    id,cam = pattern.search(person).groups()
    if(count[int(id)]<18 and count[int(id)] > 15):
      if(True):
        if os.path.exists('/content/persons/'+id) == False:
          os.makedirs('../content/persons/'+id)
        shutil.move(current_path+'/'+person, '../content/persons/'+id+'/'+person)

      else:
        if os.path.exists('/content/persons_test/'+id+'/'+cam) == False:
          os.makedirs('../content/persons_test/'+id+'/'+cam)
        shutil.move(current_path+'/'+person, '../content/persons_test/'+id+'/'+cam+'/'+person)    

In [ ]:
persons_imgs = []
for subdir, dirs, files in os.walk('/content/persons'):

  for file in files:
    filepath = subdir + os.sep + file
    if filepath.endswith(".jpg"):
      persons_imgs.append(filepath)

In [ ]:
for i in range(20):
  selected = random.choice(persons_imgs)
  selected_img = selected.split('/')
  selected_img = selected_img[-1]
  id,cam = pattern.search(selected_img).groups()
  if os.path.exists('/content/persons_test') == False:
      os.makedirs('../content/persons_test')
  shutil.move(selected, '../content/persons_test/'+selected_img)   

In [ ]:
root_dir = '../content/persons/'
categories = [[folder, os.listdir(root_dir + folder)] for folder in os.listdir(root_dir)]
print(len(categories))

In [ ]:
class Person_reid(Dataset):
    def __init__(self, categories, root_dir, setSize, transform=None):
        self.categories = categories
        self.root_dir = root_dir
        self.transform = transform
        self.setSize = setSize
    def __len__(self):
        return self.setSize
    def __getitem__(self, idx):
        img1 = None
        img2 = None
        label = None
        if idx % 2 == 0:
            category = random.choice(categories)
            imgDir = root_dir + category[0]
            img1Name = random.choice(os.listdir(imgDir))
            img2Name = random.choice(os.listdir(imgDir))
            img1 = Image.open(imgDir + '/' + img1Name)
            img2 = Image.open(imgDir + '/' + img2Name)
            label = 1.0
        else:
            category1, category2 = random.choice(categories), random.choice(categories)
            category1, category2 = random.choice(categories), random.choice(categories)

            imgDir1, imgDir2 = root_dir + category1[0], root_dir + category2[0]
            img1Name = random.choice(os.listdir(imgDir1))
            img2Name = random.choice(os.listdir(imgDir2))
            while img1Name == img2Name:
                img2Name = random.choice(os.listdir(imgDir2))
            label = 0.0
            img1 = Image.open(imgDir1 + '/' + img1Name)
            img2 = Image.open(imgDir2 + '/' + img2Name)

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2, torch.from_numpy(np.array([label], dtype=np.float32))

In [ ]:
dataSize = 12000 # self-defined dataset size
TRAIN_PCT = 0.8 # percentage of entire dataset for training
train_size = int(dataSize * TRAIN_PCT)
val_size = dataSize - train_size

transformations = transforms.Compose(
    [
     transforms.ToTensor()
    ]) 

market_id_dataset = Person_reid(categories, root_dir, dataSize, transformations)
train_set, val_set = random_split(market_id_dataset, [train_size, val_size])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=16, shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, 3) 
        self.conv2 = nn.Conv2d(64, 128, 2)  
        self.conv3 = nn.Conv2d(128, 128, 2)
        self.conv4 = nn.Conv2d(128, 256, 2)
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256*14*6, 4096)
        self.fcOut = nn.Linear(4096, 1)
        self.sigmoid = nn.Sigmoid()

    
    def convs(self, x):

        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, (2,2))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, (2,2))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, (2,2))
        x = F.relu(self.bn4(self.conv4(x)))
        return x

    def forward(self, x1, x2):
        x1 = self.convs(x1)
        x1 = x1.view(x1.shape[0],-1)
        x1 = self.sigmoid(self.fc1(x1))

        
        x2 = self.convs(x2)
        x2 = x2.view(x1.shape[0],-1)
        x2 = self.sigmoid(self.fc1(x2))

        x = torch.abs(x1 - x2)
        x = self.fcOut(x)
        return x

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
siameseBaseLine = Net()
siameseBaseLine = siameseBaseLine.to(device)

def count_parameters(model):
    temp = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'The model architecture:\n\n', model.parameters)
    print(f'\nThe model has {temp:,} trainable parameters')
    
count_parameters(siameseBaseLine)

In [ ]:
def save_checkpoint(save_path, model, optimizer, val_loss):
    if save_path==None:
        return
    save_path = save_path 
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'val_loss': val_loss}

    torch.save(state_dict, save_path)

    print(f'Model saved to ==> {save_path}')

def load_checkpoint(model, optimizer):
    save_path = f'siameseNet-batchnorm50.pt'
    state_dict = torch.load(save_path)
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    val_loss = state_dict['val_loss']
    print(f'Model loaded from <== {save_path}')
    
    return val_loss

In [ ]:
def train(model, train_loader, val_loader, num_epochs, criterion, save_name):
    best_val_loss = float("Inf") 
    train_losses = []
    val_losses = []
    cur_step = 0
    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        print("Starting epoch " + str(epoch+1))

        for img1, img2, labels in train_loader:
            # Forward
            img1 = img1.to(device)
            img2 = img2.to(device)
            labels = labels.to(device)
            outputs = model(img1, img2)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        val_running_loss = 0.0
        with torch.no_grad():
            model.eval()
            for img1, img2, labels in val_loader:
                img1 = img1.to(device)
                img2 = img2.to(device)
                labels = labels.to(device)
                outputs = model(img1, img2)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
        avg_val_loss = val_running_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        print('Epoch [{}/{}],Train Loss: {:.4f}, Valid Loss: {:.8f}'
            .format(epoch+1, num_epochs, avg_train_loss, avg_val_loss))
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            save_checkpoint(save_name, model, optimizer, best_val_loss)
    
    print("Finished Training")  
    return train_losses, val_losses  

# evaluation metrics
def eval(model, test_loader):
    with torch.no_grad():
        model.eval()
        correct = 0
        print('Starting Iteration')
        count = 0
        for mainImg, imgSets, label in test_loader:
            mainImg = mainImg.to(device)
            predVal = 0
            pred = -1
            for i, testImg in enumerate(imgSets):
                testImg = testImg.to(device)
                output = model(mainImg, testImg)
                if output > predVal:
                    pred = i
                    predVal = output
            label = label.to(device)
            if pred == label:
                correct += 1
            count += 1
            if count % 20 == 0:
                print("Current Count is: {}".format(count))
                print('Accuracy on n way: {}'.format(correct/count))

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(siameseBaseLine.parameters(), lr = 0.001)
num_epochs = 80
criterion = nn.BCEWithLogitsLoss()
save_path = 'siameseNet.pt'
train_losses, val_losses = train(siameseBaseLine, train_loader, val_loader, num_epochs, criterion, save_path)

In [ ]:
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label="Validation Loss")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()

In [ ]:
trans = transforms.Compose([
            transforms.ToTensor(),
        ])

In [ ]:
train_persons = []
for subdir, dirs, files in os.walk('/content/persons'):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".jpg"):
            train_persons.append(filepath)

In [ ]:
len(train_persons)

In [ ]:
query_path = '/content/persons_test/0482_c1s2_053121_01.jpg'
query = Image.open(query_path)
query = trans(query)
query = query.unsqueeze(0)
query = query.to(device)

scores = []
ids = []

for person in train_persons:
  img = person.split('/')
  img = img[-1]
  id,cam = pattern.search(img).groups()
  match = Image.open(person)
  match = trans(match)

  match = match.unsqueeze(0)
  match = match.to(device)

  outputs = siameseBaseLine(query, match)
  out = outputs.cpu().detach().numpy()
  
  ids.append(person)
  scores.append(out[0])


In [ ]:
img1 = cv2.imread(query_path)
img2 = cv2.imread(ids[scores.index(max(scores))])

cv2_imshow(img1)
cv2_imshow(img2)